In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=3,4,5

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=3,4,5


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
from ganime.data.base import load_dataset
from ganime.model.p2p.p2p_v3 import P2P
from ganime.visualization.videos import display_videos
import tensorflow as tf
from tqdm import tqdm
tf.get_logger().setLevel('ERROR')

In [4]:
train_ds, test_ds, input_shape = load_dataset("moving_mnist", "../../data", batch_size=64)

2022-05-04 03:01:49.956995: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-04 03:01:50.947692: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14264 MB memory:  -> device: 0, name: NVIDIA RTX A4000, pci bus id: 0000:61:00.0, compute capability: 8.6
2022-05-04 03:01:50.948623: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 14264 MB memory:  -> device: 1, name: NVIDIA RTX A4000, pci bus id: 0000:81:00.0, compute capability: 8.6
2022-05-04 03:01:50.949311: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:2 w

In [5]:
strategy = tf.distribute.MirroredStrategy()

In [26]:
with strategy.scope():
    model = P2P(g_dim=128, z_dim=64)

In [27]:
model.encoder.summary(line_length=200)

Model: "encoder"
________________________________________________________________________________________________________________________________________________________________________________________________________
 Layer (type)                                                                             Output Shape                                                                    Param #                       
 input_9 (InputLayer)                                                                     [(None, 2, 64, 64, 1)]                                                          0                             
                                                                                                                                                                                                        
 time_distributed_33 (TimeDistributed)                                                    (None, 2, 32, 32, 64)                                                           1088     

In [28]:
model.decoder.summary(line_length=200)

Model: "decoder"
________________________________________________________________________________________________________________________________________________________________________________________________________
 Layer (type)                                                                             Output Shape                                                                    Param #                       
 input_10 (InputLayer)                                                                    [(None, 20, 64)]                                                                0                             
                                                                                                                                                                                                        
 dense_19 (Dense)                                                                         (None, 20, 512)                                                                 33280    

In [37]:
with strategy.scope():
    model.compile(optimizer=tf.keras.optimizers.Adam())
history = model.fit(train_ds, epochs=100)

2022-05-02 10:33:27.551964: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:766] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_1"
op: "TensorSliceDataset"
input: "Placeholder/_0"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_UINT8
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 9000
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\024TensorSliceDataset:0"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 20
        }
        dim {
          size: 64
        }
        dim {
          size: 64
        }
        dim {
          size: 1
        }
      }
    }
  }
}



Epoch 1/100
141/141 [==============================] - 18s 63ms/step - loss: 116.1636 - reconstruction_loss: 114.4281 - kl_loss: 121.2691
Epoch 2/100
141/141 [==============================] - 9s 63ms/step - loss: 113.5409 - reconstruction_loss: 113.3631 - kl_loss: 124.6129
Epoch 3/100
141/141 [==============================] - 9s 63ms/step - loss: 112.2681 - reconstruction_loss: 112.3365 - kl_loss: 125.6535
Epoch 4/100
141/141 [==============================] - 9s 63ms/step - loss: 111.6498 - reconstruction_loss: 111.6686 - kl_loss: 126.2189
Epoch 5/100
141/141 [==============================] - 9s 63ms/step - loss: 110.9845 - reconstruction_loss: 111.2042 - kl_loss: 126.0490
Epoch 6/100
141/141 [==============================] - 9s 64ms/step - loss: 110.6015 - reconstruction_loss: 110.5734 - kl_loss: 125.9260
Epoch 7/100
141/141 [==============================] - 9s 63ms/step - loss: 109.8126 - reconstruction_loss: 109.9171 - kl_loss: 125.1517
Epoch 8/100
141/141 [===================

In [7]:
train_ds_iterator = train_ds.as_numpy_iterator()
x = next(train_ds_iterator)

2022-05-04 03:02:41.629984: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


In [8]:
generated = model(x[0])

2022-05-04 03:02:46.213277: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8101
2022-05-04 03:02:47.287414: I tensorflow/stream_executor/cuda/cuda_blas.cc:1760] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


In [9]:
display_videos(generated, n_rows=1, n_cols=5)

In [10]:
display_videos(x[1], n_rows=1, n_cols=5)

In [11]:
test_ds_iterator = test_ds.as_numpy_iterator()
x = next(test_ds_iterator)

In [12]:
generated = model(x[0])
display_videos(generated, n_rows=1, n_cols=5)

In [13]:
display_videos(x[1], n_rows=1, n_cols=5)

In [6]:
model = tf.saved_model.load("B-VAE_LSTM_longer")